## 0. Requirement

In [1]:
## jalankan ini dahulu sebelum running
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
## global
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup #scraping web statis
import time
from tqdm import tqdm #info progress

## selenium, scraping web dinamis
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import quote_plus #parsing string "spasi" menjadi "+"

In [3]:
## check if there is a response, if it's 200, we are good to go
s = requests.Session()
url = 'https://www.tempo.co/search?q=makan+bergizi+gratis&page=1'
response = s.get(url)
print(response.status_code)

200


## 1. Scraping satu artikel

In [ ]:
##Fungsi melakukan scraping data 1 halaman Tempo, masukkan string url
def scrape_tempo(url: str) -> pd.DataFrame: 
    ## Inisiasi dictionary hasil
    hasil = {}

    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        ##Judul
        judul = soup.find('h1', class_='text-[26px] font-bold leading-[122%] text-neutral-1200')
        hasil['judul'] = judul.get_text(strip=True) if judul else np.nan

        ## Sub judul
        sub_judul = soup.find('div', class_='font-roboserif leading-[156%] text-neutral-1100')
        hasil['sub_judul'] = sub_judul.get_text(strip=True) if sub_judul else np.nan

        ## Isi berita
        isi_paragraf = []
        isi_berita = soup.find_all('div', id='content-wrapper', class_='max-lg:container xl')

        for i in isi_berita:
            paragraf = i.find_all('p')
            for p in paragraf:
                teks = p.get_text(strip=True)
                if teks:  #menambahkan teks bila ada
                    isi_paragraf.append(teks)
        ringkasan = '\n\n'.join(isi_paragraf)
        hasil['isi'] = ringkasan if ringkasan else np.nan

        ## Tanggal & Jam publikasi
        tanggal_publikasi = soup.find('p', class_='text-neutral-900 text-sm')
        if tanggal_publikasi:
            waktu = tanggal_publikasi.get_text(strip=True)
            if '|' in waktu:
                tanggal, jam = [part.strip() for part in waktu.split('|')]
                hasil['tanggal'] = tanggal
                hasil['jam'] = jam
            else:
                hasil['tanggal'] = waktu
                hasil['jam'] = np.nan
        else:
            hasil['tanggal'] = np.nan
            hasil['jam'] = np.nan

        ## Kategori
        kategori = soup.find('span', class_='text-sm font-medium text-primary-main')
        hasil['kategori'] = kategori.get_text(strip=True) if kategori else np.nan

        ## Link
        hasil['link'] = url 

    except Exception as e:
        print(f"Terjadi kesalahan saat scraping: {e}")
        return None

    ## Kembalikan juga sebagai DataFrame
    df = pd.DataFrame([hasil])
    # print('selesai scraping')
    return df

In [5]:
url = 'https://www.tempo.co/ekonomi/potensi-masalah-dari-rencana-pemerintah-ubah-lapas-jadi-perumahan-1533913'
df_hasil = scrape_tempo(url)
df_hasil

,judul,sub_judul,isi,tanggal,jam,kategori,link
0,Potensi Masalah dari Rencana Pemerintah Ubah L...,Rencana ini berpotensi melanggar hak asasi par...,"TEMPO.CO,Jakarta- Menteri Perumahan dan Kawasa...",24 Mei 2025,21.00 WIB,Bisnis,https://www.tempo.co/ekonomi/potensi-masalah-d...


In [6]:
df_hasil.to_csv('./files/tempo_satu_artikel.csv', index=False, encoding='utf-8-sig')

## 2. Crawling URLs

In [ ]:
##Fungsi melakukan scraping URL Tempo, masukkan string keyword dan max halaman
def scrape_tempo_search_selenium(kata_kunci: str, halaman: int) -> pd.DataFrame: 
    # Set User-Agent
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
    opts = Options()
    opts.add_argument(f"user-agent={user_agent}")
    opts.add_argument("--headless")
    opts.add_argument("--no-sanbox")
    opts.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
        
    # Parameter input
    keyword = kata_kunci #contoh: "makan bergizi gratis"
    max_pages = halaman #contoh: 2
    results = []

    # Loop halaman
    for page in tqdm(range(1, max_pages + 1)):
        # print(f"Scraping page {page}...")

        # Format URL pencarian
        encoded_query = quote_plus(keyword)
        url = f"https://www.tempo.co/search?q={encoded_query}&page={page}"
        
        driver.get(url)
        time.sleep(10)

        try:
            container = driver.find_element("css selector", "div.flex.flex-col.divide-y.divide-neutral-500")
            beritas = container.find_elements("css selector", "figure.flex.flex-row.gap-3.py-4.container.lg\\:mx-0.lg\\:px-0")
            for berita in beritas:
                try:
                    a = berita.find_element("tag name", "a")
                    p = berita.find_element("tag name", "p")
                    results.append({
                        "judul": p.text,
                        "link": a.get_attribute("href")
                    })
                except Exception as e:
                    print("Skip 1 berita:", e)

        except Exception as e:
            print("Skip page:", e)

    driver.quit()

    # Simpan ke DataFrame
    df = pd.DataFrame(results)
    df.to_csv("./files/tempo_links.csv", index=False, encoding='utf-8-sig')
    print("Selesai. Total berita:", len(df))
    return df

In [8]:
scrape_tempo_search_selenium("makan bergizi gratis", 10)

100%|██████████| 10/10 [01:57<00:00, 11.78s/it]


Selesai. Total berita: 100


,judul,link
0,Seknas Fitra: Makan Bergizi Gratis Belum Dongk...,https://www.tempo.co/ekonomi/seknas-fitra-maka...
1,"Hingga 21 Mei, Anggaran Makan Bergizi Gratis S...",https://www.tempo.co/ekonomi/hingga-21-mei-ang...
2,Korban Keracunan MBG Lebih Besar dari Klaim Pr...,https://www.tempo.co/politik/korban-keracunan-...
3,"Sejumlah Titik Kritis dalam Program MBG, Menur...",https://www.tempo.co/politik/sejumlah-titik-kr...
4,Bos BGN Membantah Raffi Ahmad Kelola 300 Dapur...,https://www.tempo.co/ekonomi/bos-bgn-membantah...
...,...,...
95,Penerima Manfaat Makan Bergizi Gratis hingga M...,https://www.tempo.co/ekonomi/penerima-manfaat-...
96,Peluncuran Dapur Satuan Pelayanan Pemenuhan Gizi,https://www.tempo.co/foto/arsip/peluncuran-dap...
97,Cari Sebab Penerimaan Pajak Anjlok: Salah Satu...,https://www.tempo.co/ekonomi/cari-sebab-peneri...
98,Diskon Tarif Listrik Dinilai sebagai Kebijakan...,https://www.tempo.co/ekonomi/diskon-tarif-list...


## 3. Scraping artikel dalam URLs

In [ ]:
##Fungsi melakukan scraping data dari hasil crawling URL Tempo, masukkan csv
def scrape_tempo_dari_csv(path_csv: str) -> pd.DataFrame:
    # Membaca CSV hasil scraping link
    df_links = pd.read_csv(path_csv)

    # Memastikan kolom 'link' ada
    if 'link' not in df_links.columns:
        raise ValueError("CSV tidak mengandung kolom 'link'.")

    hasil_semua = []
    for i, row in tqdm(df_links.iterrows(), total=len(df_links)):
        url = row['link']
        df_artikel = scrape_tempo(url) #memanggil dan menjalankan fungsi scrape_tempo satu artikel
        if df_artikel is not None:
            hasil_semua.append(df_artikel)

    # Gabungkan semua DataFrame
    if hasil_semua:
        df_final = pd.concat(hasil_semua, ignore_index=True)
        df_final.to_csv('./files/tempo_semua_artikel.csv', index=False, encoding='utf-8-sig')
        print("Selesai menyimpan semua artikel.")
        return df_final
    else:
        print("Tidak ada artikel yang berhasil di-scrape.")
        return pd.DataFrame()

In [10]:
df = scrape_tempo_dari_csv("./files/tempo_links.csv")
df.head(5)

100%|██████████| 100/100 [02:37<00:00,  1.57s/it]

Selesai menyimpan semua artikel.


,judul,sub_judul,isi,tanggal,jam,kategori,link
0,Seknas Fitra: Makan Bergizi Gratis Belum Dongk...,Karena memaksakan Makan Bergizi Gratis pemerin...,"TEMPO.CO,Jakarta-Sekretariat Nasional Forum In...",24 Mei 2025,17.24 WIB,Bisnis,https://www.tempo.co/ekonomi/seknas-fitra-maka...
1,"Hingga 21 Mei, Anggaran Makan Bergizi Gratis S...",Hingga 21 Mei 2025 kementerian keuangan telah ...,"TEMPO.CO,Jakarta- Wakil Menteri Keuangan Suaha...",23 Mei 2025,16.11 WIB,Bisnis,https://www.tempo.co/ekonomi/hingga-21-mei-ang...
2,Korban Keracunan MBG Lebih Besar dari Klaim Pr...,Tim Cek Fakta Tempo menemukan angka keracunan ...,"TEMPO.CO,Jakarta-PRESIDEN Prabowo Subianto men...",23 Mei 2025,14.10 WIB,Politik,https://www.tempo.co/politik/korban-keracunan-...
3,"Sejumlah Titik Kritis dalam Program MBG, Menur...",Kepala BPOM mendorong keterlibatan instansinya...,KOMISI IX DPR mendesak Badan Pengawas Obat dan...,23 Mei 2025,10.13 WIB,Politik,https://www.tempo.co/politik/sejumlah-titik-kr...
4,Bos BGN Membantah Raffi Ahmad Kelola 300 Dapur...,Kabar Raffi Ahmad mendapatkan jatah mengelola ...,"TEMPO.CO,Jakarta- Kepala Badan Gizi Nasional (...",22 Mei 2025,19.30 WIB,Bisnis,https://www.tempo.co/ekonomi/bos-bgn-membantah...


## Next: Analisis Sentimen